### Export annotations from Labelbox to GCP storage

In [1]:
import os, sys
sys.path.insert(0, os.path.abspath('..'))

In [2]:
%load_ext autoreload
%autoreload 2

In [4]:
%reload_ext autoreload
from utils import gcp_utils
from utils.data_management import get_annotation_path
from utils.labelbox_utils import get_annotation_objects_from_data_row_export, get_geojson_fc_from_annotation_objects

In [6]:
# Load environment variables from .env.
# Alternatively, manually set environment variables.

from dotenv import load_dotenv
load_dotenv()

True

In [7]:
from google.cloud import storage
from labelbox import Client as LabelboxClient

GCP_PROJECT_NAME = "gee-sand"
BUCKET_NAME = "sand_mining"
PROJECT_ID = 'cljoqdjei070j0729f20hc5sx'
LABELBOX_API_KEY = os.getenv('LABELBOX_API_KEY')

gcp_client = storage.Client(project=GCP_PROJECT_NAME)
labelbox_client = LabelboxClient(api_key=LABELBOX_API_KEY)

In [36]:
project = labelbox_client.get_project(PROJECT_ID)
export = project.export_v2()

export.wait_till_done()
if export.errors:
  print("Errors:")
  print(export.errors)
else:
  print("Finished without errors.")
  export_full_project = export.result

Finished without errors.


In [49]:
from pprint import pprint

for data_row_export in export_full_project:
    global_key_data_row = data_row_export['data_row']['global_key']

    # Hack. With this particular data row, there is something weird in Label.
    # Maybe consensus was turned on when the datarow was queued. I've been unable to reverse it.
    # Let's just skip this data row. 
    if global_key_data_row == "Sone_Rohtas_84-21_24-91_2022-03-31_rgb.tif":
        continue

    print(f"Processing annotations for {global_key_data_row}")
    annotated_objects = get_annotation_objects_from_data_row_export(data_row_export)
    
    geojson_feature_collection = get_geojson_fc_from_annotation_objects(annotated_objects)
    annotation_destination_path = get_annotation_path(global_key_data_row)
    gcp_utils.upload_json(gcp_client, geojson_feature_collection, annotation_destination_path)
    

Processing annotations for Ken_Banda_80-35_25-68_2022-03-29_rgb.tif
Processing annotations for Ken_Banda_80-35_25-68_2022-12-29_rgb.tif
Processing annotations for Ken_Banda_80-35_25-68_2022-03-09_rgb.tif
Processing annotations for Ken_Banda_80-35_25-68_2022-04-08_rgb.tif
Processing annotations for Ken_Banda_80-35_25-68_2022-05-28_rgb.tif
Processing annotations for Sone_Rohtas_84-21_24-91_2022-04-30_rgb.tif
Processing annotations for Sone_Rohtas_84-21_24-91_2022-11-26_rgb.tif
Processing annotations for Sone_Rohtas_84-21_24-91_2022-05-20_rgb.tif
Processing annotations for Sone_Rohtas_84-21_24-91_2022-03-01_rgb.tif
Processing annotations for Sone_Rohtas_84-21_24-91_2022-12-16_rgb.tif
Processing annotations for Kathajodi_Cuttack_85-85_20-44_2022-05-17_rgb.tif
Processing annotations for Kathajodi_Cuttack_85-85_20-44_2022-03-03_rgb.tif
Processing annotations for Kathajodi_Cuttack_85-85_20-44_2022-05-02_rgb.tif
Processing annotations for Kathajodi_Cuttack_85-85_20-44_2022-12-28_rgb.tif
Proces